#Approach 1 Finetuning GPT-2

#Loading Dataset

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/qa_dataset.csv')

In [ ]:
pip install transformers torch datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

#Preprocessing and Arguments

In [ ]:
#df = pd.read_json("hf://datasets/toughdata/quora-question-answer-dataset/Quora-QuAD.jsonl", lines=True)
df.head()

# Preprocess your data
df['text'] = df.apply(lambda row: f"Question: {row['question']} Answer: {row['answer']}", axis=1)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_dict({'text': df['text'].tolist()})

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a pad token (use eos_token as pad_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the data
def tokenize_function(examples):
    encodings = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
    encodings['labels'] = encodings['input_ids']
    return encodings

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load the model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Resize token embeddings to include the new pad token
model.resize_token_embeddings(len(tokenizer))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.005,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
)

# Initialize Trainer
trainer = Trainer(
    model=model,                         # the instantiated model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_dataset      # training dataset
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/56402 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

#Training the Model

In [ ]:
# Train the model
trainer.train()
model.save_pretrained('./gpt2-finetuned')
tokenizer.save_pretrained('./gpt2-finetuned')


Step,Training Loss
100,17.775000
200,1.491100
300,1.262600
400,1.228600
500,1.104200
600,1.183100
700,1.201200
800,1.164200
900,1.230200
1000,1.098600


('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

#Saving the model

In [ ]:
model.save_pretrained('./gpt2-finetuned')
tokenizer.save_pretrained('./gpt2-finetuned')

('./gpt2-finetuned/tokenizer_config.json',
 './gpt2-finetuned/special_tokens_map.json',
 './gpt2-finetuned/vocab.json',
 './gpt2-finetuned/merges.txt',
 './gpt2-finetuned/added_tokens.json')

In [ ]:
import shutil
shutil.make_archive('/content/gpt2-finetuned', 'zip', './gpt2-finetuned')


'/content/gpt2-finetuned.zip'

In [ ]:
from google.colab import files
files.download('/content/gpt2-finetuned.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shutil.move('/content/gpt2-finetuned.zip', '/content/drive/My Drive/gpt2-finetuned.zip')

'/content/drive/My Drive/gpt2-finetuned.zip'

In [ ]:
pip install torch

In [ ]:
import torch

In [ ]:

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./gpt2-finetuned')
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-finetuned')

# Set pad token ID
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# Input text
input_text = "How can I use proxy?"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Create attention mask (all ones for no padding, as pad_token_id is same as eos_token_id)
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

# Generate a response with adjusted parameters
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=150,                # Increase max_length for a more complete response
    num_beams=5,                   # Use beam search for better quality
    no_repeat_ngram_size=2,        # Prevent repetitive n-grams
    temperature=0.7,               # Adjust randomness
    top_p=0.9,                     # Nucleus sampling for coherent text
    do_sample=True,                # Enable sampling for using top_p
    early_stopping=True            # Stop generation early if EOS token is detected
)

# Decode the response
response = tokenizer.decode(output[0], skip_special_tokens=True)

print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How can I use proxy? Answer: There are several ways to use a proxy server. Here are a few popular options:
Browser extensions: Some popular browsers allow you to access your website from a web browser, such as Chrome, Firefox, Safari, or Internet Explorer.Proxy servers: Websites that use proxies to connect to the internet can be hosted on a variety of proxy servers, including Google, Microsoft, and others. These proxies are typically used to provide access to websites or services that are not accessible to other users.Virtual private networks (VPNs): VPNs are networks that allow users to browse and access websites anonymously. They allow websites to communicate with each other without the need for a third-party intermediary like a VPN provider.



#Creating an interface

In [ ]:
pip install gradio

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
import gradio as gr
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./gpt2-finetuned')
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-finetuned')

# Set pad token ID
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

def generate_response(input_text):
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Create attention mask (all ones for no padding, as pad_token_id is same as eos_token_id)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    # Generate a response with adjusted parameters
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=150,                # Increase max_length for a more complete response
        num_beams=5,                   # Use beam search for better quality
        no_repeat_ngram_size=2,        # Prevent repetitive n-grams
        temperature=0.7,               # Adjust randomness
        top_p=0.9,                     # Nucleus sampling for coherent text
        do_sample=True,                # Enable sampling for using top_p
        early_stopping=True            # Stop generation early if EOS token is detected
    )

    # Decode the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Create a Gradio interface
interface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(label="Enter your question:", placeholder="How can I use proxy?", lines=2),
    outputs=gr.Textbox(label="Response"),
    title="GPT-2 Question Answering",
    description="Ask a question and get an answer from the fine-tuned GPT-2 model."
)

# Launch the interface
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e8af4fa20a0a4854cb.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


#Evaluating Model

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [ ]:


import evaluate
from datasets import load_metric

# Load and preprocess evaluation dataset
df_eval = df
df_eval = df_eval.sample(100)  # Sample a smaller subset for evaluation
df_eval['text'] = df_eval.apply(lambda row: f"Question: {row['question']} Answer: {row['answer']}", axis=1)



In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=a176620132828f92034b39ba13b34d7268fddb1bc5efd6ba6f8b29d100fdf61a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_metric
import pandas as pd
from sklearn.metrics import f1_score

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('./gpt2-finetuned')
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-finetuned')

# Set pad token ID
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

def generate_response(input_text):
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Create attention mask (all ones for no padding, as pad_token_id is same as eos_token_id)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    # Generate a response with adjusted parameters
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=150,                # Set max_length to 150
        num_beams=5,                   # Use beam search for better quality
        no_repeat_ngram_size=2,        # Prevent repetitive n-grams
        temperature=0.7,               # Adjust randomness
        top_p=0.9,                     # Nucleus sampling for coherent text
        do_sample=True,                # Enable sampling for using top_p
        early_stopping=True            # Stop generation early if EOS token is detected
    )

    # Decode the response
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Ensure the response is cut to 150 tokens if necessary
    response_tokens = response.split()  # Tokenize by whitespace
    if len(response_tokens) > 150:
        response = ' '.join(response_tokens[:150])  # Truncate to 150 tokens

    return response

# Example DataFrame (make sure to load or create df_eval)
# For example purposes, here's a dummy DataFrame
data = {'text': ["Question: What is the capital of France? Answer: Paris"]}
df_eval = pd.DataFrame(data)

# Generate responses for the dataset
df_eval['generated_answer'] = df_eval['text'].apply(generate_response)

# Load metrics
rouge = load_metric('rouge')
bleu = load_metric('bleu')

# Define the function to compute F1-score
def compute_f1_score(predictions, references):
    # Tokenize answers for comparison (assuming both are lists of words)
    # For exact matching, consider using exact match metrics or further processing
    f1 = f1_score(references, predictions, average='weighted')
    return f1

# Compute metrics
def compute_metrics(df):
    predictions = df['generated_answer'].tolist()
    references = df['text'].apply(lambda x: x.split(' Answer: ')[-1]).tolist()  # Extract reference answers

    # Compute ROUGE
    rouge_results = rouge.compute(predictions=predictions, references=references)

    # Compute BLEU
    bleu_results = bleu.compute(predictions=[p.split() for p in predictions], references=[[r.split()] for r in references])

    # Compute F1-score (assuming answers are tokenized into words)
    f1_score_value = compute_f1_score(predictions, references)

    return {
        'ROUGE': rouge_results,
        'BLEU': bleu_results,
        'F1-score': f1_score_value
    }

# Calculate metrics
metrics = compute_metrics(df_eval)
print(metrics)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
<ipython-input-21-32c14ccbf8de>:54: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


The repository for bleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
{'ROUGE': {'rouge1': AggregateScore(low=Score(precision=0.008333333333333333, recall=1.0, fmeasure=0.01652892561983471), mid=Score(precision=0.008333333333333333, recall=1.0, fmeasure=0.01652892561983471), high=Score(precision=0.008333333333333333, recall=1.0, fmeasure=0.01652892561983471)), 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)), 'rougeL': AggregateScore(low=Score(precision=0.008333333333333333, recall=1.0, fmeasure=0.01652892561983471), mid=Score(precision=0.008333333333333333, recall=1.0, fmeasure=0.01652892561983471), high=Score(precision=0.00833

The output will include values for ROUGE, BLEU, and F1-score, which will help you understand the quality of your model’s responses in comparison to the reference answers.

ROUGE measures the overlap of n-grams between the generated and reference texts.
BLEU measures the precision of n-grams in the generated text relative to the reference.
F1-score evaluates the accuracy of the generated text in terms of token-level matches.